In [1]:
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv("data/train_combine.csv")
train.head()

,id,author,geometry,pressure [MPa],mass_flux [kg/m2-s],x_e_out [-],D_e [mm],D_h [mm],length [mm],chf_exp [MW/m2]
0,0,Thompson,tube,7.00,3770.0,0.1754,NaN,10.8,432.0,3.6
1,1,Thompson,tube,NaN,6049.0,-0.0416,10.3,10.3,762.0,6.2
2,2,Thompson,NaN,13.79,2034.0,0.0335,7.7,7.7,457.0,2.5
3,3,Beus,annulus,13.79,3679.0,-0.0279,5.6,15.2,2134.0,3.0
4,5,NaN,NaN,17.24,3648.0,-0.0711,NaN,1.9,696.0,3.6


In [3]:
y = train['x_e_out [-]']
X = train.drop(columns=['id','x_e_out [-]'])

In [4]:
X

,author,geometry,pressure [MPa],mass_flux [kg/m2-s],D_e [mm],D_h [mm],length [mm],chf_exp [MW/m2]
0,Thompson,tube,7.00,3770.0,NaN,10.8,432.0,3.6
1,Thompson,tube,NaN,6049.0,10.3,10.3,762.0,6.2
2,Thompson,NaN,13.79,2034.0,7.7,7.7,457.0,2.5
3,Beus,annulus,13.79,3679.0,5.6,15.2,2134.0,3.0
4,NaN,NaN,17.24,3648.0,NaN,1.9,696.0,3.6
...,...,...,...,...,...,...,...,...
23089,Richenderfer,plate,1.01,1500.0,15.0,120.0,10.0,9.4
23090,Richenderfer,plate,1.01,1500.0,15.0,120.0,10.0,10.4
23091,Richenderfer,plate,1.01,2000.0,15.0,120.0,10.0,10.8
23092,Richenderfer,plate,1.01,2000.0,15.0,120.0,10.0,10.9


In [5]:
def preprocess_data(X):
    X['chf_exp [MW/m2]'] = np.log(X['chf_exp [MW/m2]'])
    X['D_e [mm]'] = np.log(X['D_e [mm]'])
    X['D_h [mm]'] = np.log(X['D_h [mm]'])
    X['length [mm]'] = np.log(X['length [mm]'])
    return X

X = preprocess_data(X)
X

,author,geometry,pressure [MPa],mass_flux [kg/m2-s],D_e [mm],D_h [mm],length [mm],chf_exp [MW/m2]
0,Thompson,tube,7.00,3770.0,NaN,2.379546,6.068426,1.280934
1,Thompson,tube,NaN,6049.0,2.332144,2.332144,6.635947,1.824549
2,Thompson,NaN,13.79,2034.0,2.041220,2.041220,6.124683,0.916291
3,Beus,annulus,13.79,3679.0,1.722767,2.721295,7.665753,1.098612
4,NaN,NaN,17.24,3648.0,NaN,0.641854,6.545350,1.280934
...,...,...,...,...,...,...,...,...
23089,Richenderfer,plate,1.01,1500.0,2.708050,4.787492,2.302585,2.240710
23090,Richenderfer,plate,1.01,1500.0,2.708050,4.787492,2.302585,2.341806
23091,Richenderfer,plate,1.01,2000.0,2.708050,4.787492,2.302585,2.379546
23092,Richenderfer,plate,1.01,2000.0,2.708050,4.787492,2.302585,2.388763


In [6]:
#from sklearn.impute import KNNImputer,SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import make_pipeline

In [7]:
scaler = StandardScaler()

In [8]:
ohe = OneHotEncoder(handle_unknown='ignore')

In [9]:
from sklearn.compose import make_column_selector, make_column_transformer

In [10]:
num_cols = make_column_selector(dtype_include='number')
cat_cols = make_column_selector(dtype_exclude='number')

In [11]:
preprocessor = make_column_transformer(
     (scaler, num_cols),
     (ohe, cat_cols))

In [12]:
import lightgbm as lgb
lgbm = lgb.LGBMRegressor(n_estimators=122,random_state = 318) #0.00545159

from sklearn.ensemble import HistGradientBoostingRegressor,VotingRegressor
hgbr = HistGradientBoostingRegressor(max_iter=750,random_state=318) #0.00546830

from catboost import CatBoostRegressor
cb = CatBoostRegressor(n_estimators=869,random_state = 318,verbose=0, objective='RMSE') #0.00548920

vr = VotingRegressor([('lgbm',lgbm),('hgbr',hgbr)]) #0.0054346
vr1 = VotingRegressor([('lgbm',lgbm),('hgbr',hgbr),('cb',cb)]) #0.0054152

In [13]:
pipe = make_pipeline(preprocessor, cb)

In [14]:
from sklearn.model_selection import cross_val_score
cross_val_score(pipe, X, y, scoring='neg_mean_squared_error').mean()

-0.005489201065864164

In [15]:
pipe_vr = make_pipeline(preprocessor, vr)

In [16]:
pipe_vr.steps

[('columntransformer',
  ColumnTransformer(transformers=[('standardscaler', StandardScaler(),
                                   <sklearn.compose._column_transformer.make_column_selector object at 0x00000277BCF7B490>),
                                  ('onehotencoder',
                                   OneHotEncoder(handle_unknown='ignore'),
                                   <sklearn.compose._column_transformer.make_column_selector object at 0x00000277BCF7B6A0>)])),
 ('votingregressor',
  VotingRegressor(estimators=[('lgbm',
                               LGBMRegressor(n_estimators=122, random_state=318)),
                              ('hgbr',
                               HistGradientBoostingRegressor(max_iter=750,
                                                             random_state=318))]))]

In [17]:
from sklearn.model_selection import GridSearchCV

params = {}
params['votingregressor__weights'] = [(1,1),(1,2),(2,1)]

grid = GridSearchCV(pipe_vr, params, scoring='neg_mean_squared_error')
%time grid.fit(X,y)

Wall time: 32.3 s


GridSearchCV(estimator=Pipeline(steps=[('columntransformer',
                                        ColumnTransformer(transformers=[('standardscaler',
                                                                         StandardScaler(),
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x00000277BCF7B490>),
                                                                        ('onehotencoder',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x00000277BCF7B6A0>)])),
                                       ('votingregressor',
                                        VotingRegressor(estimators=[('lgbm',
                                                                     LGBMRegressor(n_esti

In [18]:
grid.best_params_

{'votingregressor__weights': (1, 1)}

In [19]:
grid.best_score_

-0.0054343667593764605

- vr1得分

In [20]:
pipe_vr1 = make_pipeline(preprocessor, vr1)

In [21]:
pipe_vr1.steps

[('columntransformer',
  ColumnTransformer(transformers=[('standardscaler', StandardScaler(),
                                   <sklearn.compose._column_transformer.make_column_selector object at 0x00000277BCF7B490>),
                                  ('onehotencoder',
                                   OneHotEncoder(handle_unknown='ignore'),
                                   <sklearn.compose._column_transformer.make_column_selector object at 0x00000277BCF7B6A0>)])),
 ('votingregressor',
  VotingRegressor(estimators=[('lgbm',
                               LGBMRegressor(n_estimators=122, random_state=318)),
                              ('hgbr',
                               HistGradientBoostingRegressor(max_iter=750,
                                                             random_state=318)),
                              ('cb',
                               <catboost.core.CatBoostRegressor object at 0x00000277BCF84220>)]))]

In [22]:
from sklearn.model_selection import GridSearchCV

params = {}
params['votingregressor__weights'] = [(1,1,1),(1,2,1),(2,1,1),(1,1,2)]

grid = GridSearchCV(pipe_vr1, params, scoring='neg_mean_squared_error')
%time grid.fit(X,y)

Wall time: 2min 5s


GridSearchCV(estimator=Pipeline(steps=[('columntransformer',
                                        ColumnTransformer(transformers=[('standardscaler',
                                                                         StandardScaler(),
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x00000277BCF7B490>),
                                                                        ('onehotencoder',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x00000277BCF7B6...
                                       ('votingregressor',
                                        VotingRegressor(estimators=[('lgbm',
                                                                     LGBMRegressor(n_estimator

In [23]:
grid.best_params_

{'votingregressor__weights': (1, 1, 1)}

In [24]:
grid.best_score_

-0.0054120597861867835

- 超过了之前的score

In [25]:
#处理test值
test = pd.read_csv('data/test.csv')
X_test = test.drop(columns=['id','x_e_out [-]'])

In [26]:
X_test1 = preprocess_data(X_test)

In [27]:
result = grid.predict(X_test1)

In [28]:
result

array([ 0.00094321, -0.09500588, -0.057206  , ...,  0.02814031,
       -0.06068587, -0.01378407])

In [29]:
pd.DataFrame({'id':test.id, 'x_e_out [-]':result}).set_index('id').to_csv('sub_combine_data_lgbm_cb_hgbr_no_imputing.csv')

- 0.075537